# MODEL TRANSFORMER

## 1. Import

In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import keras_nlp
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer



## 2. Data

In [3]:
df = pd.read_csv('data/data.txt', sep=';')
df.head(3)

In [135]:
#df.Lyric.str.replace('\n'," ")
#lyrics = df.Lyric.str.split('\n')

In [136]:
#df['Seq_count'] = df.Lyric.apply(lambda x: len(str(x).split()))
#df['Seq_count'] = [len(x) for x in lyrics]

In [18]:
def clean_lyrics(lyrics):
    # séparation des paroles par groupes de mots
    lyrics = list(lyrics.split('\n'))
    lyrics_length = len(lyrics)
    word_bundle_list = []
    cleaned_lyrics =  []
    for word_bundle in lyrics :
        word_bundle = word_tokenize(word_bundle)
        word_bundle = [word.lower().encode("utf8").decode("ascii",'ignore') for word in word_bundle if word is not string.punctuation and word.isalpha()]
        word_bundle_list.append(len(word_bundle))
        cleaned_lyrics.append(' '.join(word_bundle))

    while '' in cleaned_lyrics:
        cleaned_lyrics.remove('')
    
    cleaned_lyrics = ' '.join(cleaned_lyrics)

    word_bundle_mean_length = round(np.array(word_bundle_list).mean())
    # N=4
    # cleaned_text = [cleaned_text[n:n+N] for n in range(0, len(cleaned_text), N)]
    # for seq in cleaned_text :
    #     seq = ' '.join(seq)
    #     print(seq)
    # print('sub_corpus_length :', len(cleaned_text))
    # cleaned_text = ' '.join(cleaned_text)

    return np.array([cleaned_lyrics, lyrics_length, word_bundle_mean_length], dtype='object')

In [19]:
sub_dataset = df.Lyric[:20]
corpus = pd.DataFrame([clean_lyrics(x) for x in sub_dataset], columns=['lyrics', 'bundle_of_words_length', 'words_per_bundle_mean'])

In [20]:
corpus.describe()

,bundle_of_words_length,words_per_bundle_mean
count,20.000000,20.000000
mean,32.200000,5.200000
std,10.496115,1.056309
min,16.000000,3.000000
25%,24.750000,4.750000
50%,31.500000,5.000000
75%,39.250000,6.000000
max,52.000000,7.000000


In [431]:
# def max_seq_length(corpus):
#     corpus_length = []
#     for seq in corpus :
#         for word in seq :
#             seq = seq.split()
#             corpus_length.append(len(seq))
#     round(np.array(corpus_length).mean())

In [58]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
nlp.max_length = len(corpus.lyrics.sum())
print('corpus length :', len(corpus.lyrics.sum()))

corpus length : 15107


In [120]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus, seq_length):
    train_len = seq_length
    lyric_sequences = []
    input_sequences = []
    for lyric in corpus :
        tokens = [token.text for token in nlp(corpus)]
        print(len(tokens))
        input_sequences = pad_sequences(tokens, maxlen=seq_length, padding='pre')
        print(input_sequences)
    # for i in range(train_len, len(tokens)):
    #     seq = tokens[i-train_len:i]
    #     lyric_sequences.append(seq)

    # lyric_sequences=list(itertools.chain(lyric_sequences))
    ## tokenization
    # tokenizer.fit_on_texts(lyric_sequences)
    # total_words = len(tokenizer.word_index) + 1
    # input_sequences = tokenizer.texts_to_sequences(lyric_sequences)
    # return input_sequences, total_words
    return lyric_sequences

In [122]:
seq_length = corpus.words_per_bundle_mean.max()
lyrics_length = corpus.words_per_bundle_mean.min()
input_sequences = [get_sequence_of_tokens(x, seq_length) for x in corpus.lyrics]

110


ValueError: invalid literal for int() with base 10: 'wise'

In [106]:
input_sequences

[[['wise', 'men', 'say', 'only', 'fools', 'rush', 'in'],
  ['men', 'say', 'only', 'fools', 'rush', 'in', 'but'],
  ['say', 'only', 'fools', 'rush', 'in', 'but', 'i'],
  ['only', 'fools', 'rush', 'in', 'but', 'i', 'ca'],
  ['fools', 'rush', 'in', 'but', 'i', 'ca', 'help'],
  ['rush', 'in', 'but', 'i', 'ca', 'help', 'falling'],
  ['in', 'but', 'i', 'ca', 'help', 'falling', 'in'],
  ['but', 'i', 'ca', 'help', 'falling', 'in', 'love'],
  ['i', 'ca', 'help', 'falling', 'in', 'love', 'with'],
  ['ca', 'help', 'falling', 'in', 'love', 'with', 'you'],
  ['help', 'falling', 'in', 'love', 'with', 'you', 'shall'],
  ['falling', 'in', 'love', 'with', 'you', 'shall', 'i'],
  ['in', 'love', 'with', 'you', 'shall', 'i', 'stay'],
  ['love', 'with', 'you', 'shall', 'i', 'stay', 'would'],
  ['with', 'you', 'shall', 'i', 'stay', 'would', 'it'],
  ['you', 'shall', 'i', 'stay', 'would', 'it', 'be'],
  ['shall', 'i', 'stay', 'would', 'it', 'be', 'a'],
  ['i', 'stay', 'would', 'it', 'be', 'a', 'sin'],
  ['st

In [435]:
# tokenizer = Tokenizer()

# def get_sequence_of_tokens(corpus):
#     train_len = corpus.words_per_bundle_mean.min()
#     lyric_sequences = []
#     for lyric in corpus :
#         tokens = [token.text for token in nlp(lyric)]
#         for i in range(train_len, len(tokens)):
#             seq = tokens[i-train_len:i]
#             lyric_sequences.append(seq)
#     ## tokenization
#     tokenizer.fit_on_texts(lyric_sequences)
#     total_words = len(tokenizer.word_index) + 1
#     input_sequences = tokenizer.texts_to_sequences(lyric_sequences)
#     return input_sequences, total_words

In [436]:
input_sequences, total_words = get_sequence_of_tokens(corpus)

In [437]:
def generate_padded_sequences(input_sequences):
    input_sequences=list(itertools.chain(input_sequences))
    # max_sequence_len = max([len(x) for x in input_sequences])
    # print('max sequence length :', max_sequence_len)
    #input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    #input_sequences = np.array(input_sequences, dtype='object')
    input_sequences = np.array(pad_sequences(input_sequences, padding='pre'))
    X, y = tf.convert_to_tensor(input_sequences[:-1], dtype=tf.int64), tf.convert_to_tensor(input_sequences[-1], dtype=tf.int64)
    y = to_categorical(y, num_classes=total_words)
    seq_length = X.shape[1]
    return X, y, seq_length

In [438]:
predictors, label, sequence_length = generate_padded_sequences(input_sequences)

In [439]:
print(predictors.shape, label.shape)

(100, 9) (9, 44)


## 3. Keras

In [2]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  # Limits parameters in model.

# Training
EPOCHS = 6

# Inference
NUM_TOKENS_TO_GENERATE = 80


In [3]:
keras.utils.get_file(
    origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    extract=True,
)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/")

# Load simplebooks-92 train set and filter out short lines.
raw_train_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# Load simplebooks-92 validation set and filter out short lines.
raw_val_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)

print(raw_train_ds)

282386239/282386239 [==============================] - 228s 1us/step
<ShuffleDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>


In [13]:
raw_train_ds

In [11]:
# Train tokenizer vocabulary
from keras_nlp.tokenizers.word_piece_tokenizer_trainer import compute_word_piece_vocabulary
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

ImportError: cannot import name 'pretokenize' from 'keras_nlp.tokenizers.word_piece_tokenizer' (d:\02.Pro\SIMPLON\Formation\S17.S18-NLP_Aout.Sept2022\brief_lyrics_generator\.env\lib\site-packages\keras_nlp\tokenizers\word_piece_tokenizer.py)

In [5]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

NameError: name 'vocab' is not defined

In [ ]:
# packer adds a start token
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


# Tokenize and split into train and label sequences.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

In [12]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Giving one argument only skips cross-attention.
# Output.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=['accuracy', perplexity])

TypeError: Exception encountered when calling layer "token_and_position_embedding" (type TokenAndPositionEmbedding).

in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__get_config() takes 1 positional argument but 2 were given


Call arguments received by layer "token_and_position_embedding" (type TokenAndPositionEmbedding):
  • inputs=tf.Tensor(shape=(None, None), dtype=int32)

In [ ]:
model.summary()

In [ ]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)

In [441]:
from pickle import dump, load

In [444]:
history = model.fit(
    x=predictors, 
    y=label, 
    batch_size=32, 
    epochs=10, 
    verbose=1
)

ValueError: Data cardinality is ambiguous:
  x sizes: 100
  y sizes: 9
Make sure all arrays contain the same number of samples.